In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import sys
sys.executable

'/home/peromoseq/anaconda3/envs/mmdeploy/bin/python3'

### important
- must be `module load cuda/11.7`

##### Duration: ???

In [4]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


In [5]:
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Python interpreter binary location: /home/peromoseq/anaconda3/envs/mmdeploy/bin/python3


### Get recording info (google sheets)

In [6]:
import requests
import pandas as pd
from io import BytesIO

In [7]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [8]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10
1,M04003,30,24-05-01-13-51-17-581428,2024-05-01_13-51-09,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10
2,M04002,30,24-05-01-15-37-07-757018,2024-05-01_15-36-57,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Run 2D prediction

In [9]:
for idx, recording_row in recording_df.iterrows():
    break

In [10]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [11]:
from multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt import RTMModelConverter
output_directory_tensorrt = output_directory / 'tensorrt'
output_directory_tensorrt.mkdir(parents=True, exist_ok=True)

# https://o2portal.rc.hms.harvard.edu/node/compute-g-17-148.o2.rc.hms.harvard.edu/59141/notebooks/tim_sainburg/projects/23-09-29-peromoseq/notebooks/keypoints/mmpose/24-01-11-deploy-2d-predictions-on-local/24-05-08-convert-pose-model.ipynb
rtmpose_model_name = 'rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524'
path_to_rmpose_config = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py')
path_to_rmpose_checkpoint = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth')

# https://o2portal.rc.hms.harvard.edu/node/compute-g-17-148.o2.rc.hms.harvard.edu/59141/notebooks/tim_sainburg/projects/23-09-29-peromoseq/notebooks/keypoints/mmpose/24-01-11-deploy-2d-predictions-on-local/240508-convert-detection-model-fp32.ipynb
rtmdetection_model_name = 'rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661'
path_to_rtmdetection_config = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/config.py')
path_to_rtmdetection_checkpoint = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth')

model_converter = RTMModelConverter(
    path_to_rmpose_config = path_to_rmpose_config,
    path_to_rmpose_checkpoint = path_to_rmpose_checkpoint,
    path_to_rtmdetection_config = path_to_rtmdetection_config,
    path_to_rtmdetection_checkpoint = path_to_rtmdetection_checkpoint,
    tensorrt_output_directory = output_directory_tensorrt,
    rtmdetection_model_name = rtmdetection_model_name,
    rtmpose_model_name = rtmpose_model_name,
    conda_env = "/home/peromoseq/anaconda3/envs/mmdeploy",
    is_local=True
)

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Python interpreter binary location: /home/peromoseq/anaconda3/envs/mmdeploy/bin/python3
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Using CUDA device: NVIDIA_GeForce_RTX_4070_Ti


In [12]:
output_directory_tensorrt

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt')

In [13]:
model_converter.run()

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Converting detection model to tensorrt. input: /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/NVIDIA_GeForce_RTX_4070_Ti
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Checking if tensorrt model exists at: /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/NVIDIA_GeForce_RTX_4070_Ti
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:TensorRT detection model already exists.
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Converting pose model to tensorrt. input: /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/NVIDIA_GeForce_RTX_4070_Ti
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Checking if tensorrt model 

In [14]:
output_directory_2D = output_directory/ "2D_predictions-2" 

In [15]:
output_directory_2D.mkdir(parents=True, exist_ok=True)

In [16]:
recording_row.video_location_on_o2

'/n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/'

In [17]:
video_directory = Path(recording_row.video_location_on_o2) / recording_row.video_recording_id

In [18]:
pose_checkpoint = "/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth"
pose_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py'
det_checkpoint = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth'
det_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/config.py'

In [19]:
#tensorrt_path = "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt"
rtmpose_model_name = 'rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524'
rtmdetection_model_name = 'rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661'

In [20]:
output_directory_predictions = output_directory_2D / recording_row.video_recording_id

In [21]:
tensorrt_model_directory = output_directory / "tensorrt"

In [22]:
output_directory_predictions = output_directory_2D / (recording_row.video_recording_id + '_tensorrt')

In [23]:
output_directory_predictions

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/2D_predictions-2/24-05-01-13-26-43-110846_tensorrt')

In [24]:
expected_video_length_frames = recording_row.max_video_duration_m * recording_row.samplerate

In [25]:
inferencer = Inferencer2D(
    video_directory,
    output_directory_predictions = output_directory_predictions,
    pose_estimator_config = pose_config,
    pose_estimator_checkpoint = pose_checkpoint,
    detector_config = det_config,
    detector_checkpoint = det_checkpoint,
    tensorrt_rtmdetection_model_name = rtmdetection_model_name,
    tensorrt_rtmpose_model_name = rtmpose_model_name,
    tensorrt_model_directory = tensorrt_model_directory,
    use_tensorrt = False,
    expected_video_length_frames = 90000,
)

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Using CUDA device: NVIDIA_GeForce_RTX_4070_Ti
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Using Tensorrt: False
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Init completed


In [ ]:
inferencer.run()

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Running 2D inference over videos
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Video processing incomplete, running
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Processing 12 videos


processing videos:   0%|          | 0/12 [00:00<?, ?it/s]

Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth
Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth


INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:models loaded (not tensorrt)


frames:   0%|          | 0/90000 [00:00<?, ?it/s]

08/19 12:57:12 - mmengine - WARNING - The current default scope "mmdet" is not "mmpose", `init_default_scope` will force set the currentdefault scope to "mmpose".


/home/peromoseq/anaconda3/envs/mmdeploy/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400400184/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
from mmdet.apis import init_detector
from mmpose.utils import adapt_mmdet_pipeline
from mmpose.apis import init_model as init_pose_estimator

In [ ]:
pose_estimator_config = Path(pose_config)
pose_estimator_checkpoint = Path(pose_checkpoint)
detector_config = Path(det_config)
detector_checkpoint = Path(det_checkpoint)

In [ ]:
pose_estimator = init_pose_estimator(
    pose_estimator_config.as_posix(),
    pose_estimator_checkpoint.as_posix(),
    device="cuda",
    cfg_options=dict(model=dict(test_cfg=dict(output_heatmaps=False))),
)

In [ ]:
detector = init_detector(
    detector_config.as_posix(), detector_checkpoint.as_posix(), device="cuda"
)

In [ ]:
detector.cfg = adapt_mmdet_pipeline(detector.cfg)